# Project Phase II
## Goal
Finding the features that can affect the number of accident in Calgary City.
All data provided is based on https://data.calgary.ca/browse

## Features
### Road Features
#### 1. Road Speed
    https://data.calgary.ca/Health-and-Safety/Speed-Limits-Map/rbfp-3tic
#### 2. Average Traffic Volume
    2018 (Traffic_Volumes_for_2018.csv)
#### 3. Road Signals
    a. Traffic Signals (Traffic_Signals.csv)
    b. Traffic Signs (Traffic_Signs.csv)
    c. Traffic cameras (Traffic_Camera_Locations.csv)
### Weather Features
    ● Temperature
    ● Visibility
    Ref: climate.weather.gc.ca
## Marking
    ● Analysing Data- (Visualization: 10 Marks + Conclusion: 5 Marks) (15 Marks)
    ● Visualizing speed limit (5 Marks)
    ● Visualizing Traffic heatmap (5 Marks)
    ● Project Demo (5 Marks)
    ● Total Mark: 30 Marks
## Due date 
To upload the report(Presentation Slides) and source code: 13-Aug 11:59 midnight.

## 1. Data Preparation

### 1.1 Data Cleaning
    ● Read the calgary boundary from City_Boundary_layer.csv.
    ● Draw a rectangle on Calgary map that shows the boundary of Calgary City.

In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
import geopandas as gpd
from geopandas import GeoDataFrame
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# load boundary from csv and store long/lat separately in df
import re
city_boundary_df = pd.read_csv('City_Boundary_layer.csv')
geom = city_boundary_df.iloc[0]['the_geom']
g=re.split("POLYGON", geom)[1].strip()
temp = pd.DataFrame(re.sub('[()]', '', g).split(', '))
boundary_coordinates_df=temp[0].str.split(" ", n = 1, expand = True).astype(float)
boundary_coordinates_df.columns=['Longitude', 'Latitude']
#boundary_coordinates_df.describe()

In [3]:
from shapely.geometry import Polygon
# boundaries in four directions w, e, n, s
w = boundary_coordinates_df['Longitude'].max()
e = boundary_coordinates_df['Longitude'].min()
n = boundary_coordinates_df['Latitude'].max()
s = boundary_coordinates_df['Latitude'].min()
polygon_geom = Polygon([(w, n), (w, s), (e, s), (e, n)])
crs = 'epsg:4326'
polygon = gpd.GeoDataFrame(index=[0], crs=crs, geometry=[polygon_geom])       
print(polygon.geometry)

0    POLYGON ((-113.85990 51.21243, -113.85990 50.8...
Name: geometry, dtype: geometry


In [4]:
import folium


map = folium.Map(location=[51.03011, -114.08529], zoom_start = 10)
folium.GeoJson(polygon).add_to(map)
folium.LatLngPopup().add_to(map)
map
# map.choropleth(geo_data=city_boundary_gdf['geometry'])

### 1.2 Data Merging
    ● Divide calgary to a 10x10 matrix of areas. 
      You need to investigate each area according to different features.

In [5]:

# generate 11*1 1-d array, listing longitude from w to e
x = np.linspace(w, e, num=11)[::-1]
# generate 1*11 1-d array, listing latitude from n to s
y = np.linspace(n, s, num=11)
# generate 11*11 2-d array, listing longitude and latitude separately in xv and yv
xv, yv = np.meshgrid(x, y, indexing='ij')
# https://numpy.org/doc/stable/reference/generated/numpy.meshgrid.html
# grids are named following the pattern "grid" + x + y, 
# e.g. northwest corner is named grid00 and southeast corner grid is named grid99
grid_names=[]
west_boundary=[]
east_boundary=[]
north_boundary=[]
south_boundary=[]
polygons=[]
for i in range(10):
    for j in range(10):
        # write grid names into df
        grid_names.append('grid{}{}'.format(i,j))
        # write 4 boundaries into df
        west_boundary.append(x[i])
        east_boundary.append(x[i+1])
        north_boundary.append(y[j])
        south_boundary.append(y[j+1])
        # find the nw, sw, se, ne corner coordinates and store in df
        grid_corners=[
            (xv[i][j], yv[i][j]), 
            (xv[i][j+1], yv[i][j+1]), 
            (xv[i+1][j+1], yv[i+1][j+1]), 
            (xv[i+1][j], yv[i+1][j])
        ]    
        polygons.append(Polygon(grid_corners))

grid_df = pd.DataFrame({'Grid Names':grid_names, 
                       'West Boundary':west_boundary, 
                       'East Boundary':east_boundary, 
                       'North Boundary':north_boundary, 
                       'South Boundary':south_boundary})

polygon_gdf = gpd.GeoDataFrame(crs='epsg:4326', geometry=polygons) 
grid_df = pd.concat([grid_df, polygon_gdf['geometry']], axis=1)
folium.GeoJson(polygon_gdf).add_to(map)
map

## 2. Data Aggregation
For Each area (grid) calculate the following features: (15 Marks)

    ● Average speed limit
    ● Average Traffic volume
    ● Average number of traffic cameras
    ● Number of Traffic Signals
    ● Number of Traffic Signs
    ● Daily Weather Condition
        ○ Temperature
        ○ Visibility
    ● Target: Average number of Traffic accidents
    ● Analyse the data and interpret what is the relation between the number of
    accidents and the above feature in 2018. (Use different techniques of visualizing
    data like histogram, scatter plot, line graph, heatmap to interpret your answer)

### 2.1 Analysing a specific group of data

In [6]:
speed_df = pd.read_csv('Speed_Limits.csv', usecols=['SPEED', 'multiline'])

volume_df = pd.read_csv('Traffic_Volumes_for_2018.csv', 
                        usecols=['YEAR', 'VOLUME', 'multilinestring'])
volume_df = volume_df[volume_df['YEAR']==2018].drop(columns=['YEAR'])

cameras_df = pd.read_csv('Traffic_Camera_Locations.csv', usecols=['longitude', 'latitude'])

signals_df = pd.read_csv('Traffic_Signals.csv', 
                         usecols=['longitude', 'latitude', 'Point', 'Count'])

signs_df = pd.read_csv('Traffic_Signs.csv', usecols=['BLADE_TYPE', 'POINT'])

incident_df = pd.read_csv('Traffic_Incidents.csv', usecols=['START_DT', 'Longitude', 'Latitude'])
incident_df = incident_df[incident_df['START_DT'].str.contains('2018')]
incident_df['DateTime'] = pd.to_datetime(incident_df['START_DT'])
incident_df['Date'] = incident_df['DateTime'].dt.strftime('%Y-%m-%d')
incident_df['Time'] = incident_df['DateTime'].dt.strftime('%H:%M')
incident_df=incident_df.sort_values(by=['Date']).reset_index(drop=True)

#### 2.1.1 Average Speed Limit
##### is calculated to be a road length weighted speed limit value.
1. Use x = a.intersection(b) to returns a intersected geometry of MULTILINESTRING and Polygon.
2. Use geometry_object.length to return the length of the intersected geometry.
3. (speed limit * road segment length)/(total length of all segments) in grid

In [7]:
# (speed limit * road segment length) / total length of all segments in cell
import shapely.wkt
grid_df.assign(Speed_Limit=np.nan)
i=0
for polygon in grid_df['geometry']:
    weighted_total_speed=0
    total_length=0
    j=0
    for m in speed_df['multiline']:
        # convert string m to MULTILINESTRING object MultiLineString
        MultiLineString = shapely.wkt.loads(m)
        intersection=polygon.intersection(MultiLineString)
        speed=speed_df.iloc[j, 0]
        weighted_total_speed+=speed*intersection.length
        total_length+=intersection.length
        j+=1
    grid_df.at[i, 'Speed_Limit'] = (weighted_total_speed/total_length)
    i+=1

/Users/apple/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in double_scalars


#### 2.1.2 Average Traffic Volume
##### is calculated to be a road length weighted traffic volume value.
1. Use x = a.intersection(b) to returns a intersected geometry of MULTILINESTRING and Polygon.
2. Use geometry_object.length to return the length of the intersected geometry.
3. (traffic volume * road segment length)/(total length of all segments) in grid

In [8]:
grid_df.assign(Traffic_Volume=np.nan)
i=0
for polygon in grid_df['geometry']:
    weighted_total_volume=0
    total_length=0
    j=0
    for m in volume_df['multilinestring']:
        # convert string m to MULTILINESTRING object MultiLineString
        MultiLineString = shapely.wkt.loads(m)
        intersection=polygon.intersection(MultiLineString)
        volume=volume_df.iloc[j, 0]
        weighted_total_volume+=volume*intersection.length
        total_length+=intersection.length
        j+=1
    grid_df.at[i, 'Traffic_Volume'] = (weighted_total_speed/total_length)
    i+=1

/Users/apple/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in double_scalars
  from ipykernel import kernelapp as app


#### 2.1.3 Total (not average) Number of Traffic Cameras

In [9]:
from shapely.geometry import Point, Polygon
grid_df.assign(Traffic_Cameras=np.nan)
idx=0
for polygon in grid_df['geometry']:
    count=0
    for long, lat in zip(cameras_df['longitude'], cameras_df['latitude']):
        point = Point(long, lat)
        # check if points are inside of grid polygon
        if point.within(polygon):
            count+=1
    grid_df.at[idx, 'Traffic_Cameras'] = count
    idx+=1

#### 2.1.4 Total Number of Traffic Signals

In [10]:
grid_df.assign(Traffic_Signals=np.nan)
idx=0
for polygon in grid_df['geometry']:
    count=0
    for long, lat in zip(signals_df['longitude'], signals_df['latitude']):
        point = Point(long, lat)
        if point.within(polygon):
            count+=1
    grid_df.at[idx, 'Traffic_Signals'] = count
    idx+=1

#### 2.1.5 Total Number of Traffic Signs

In [11]:
import shapely.wkt
grid_df.assign(Traffic_Signs=np.nan)
idx=0
for polygon in grid_df['geometry']:
    count=0
    for p in signs_df['POINT']:
        # convert string p to Point object point
        point = shapely.wkt.loads(p)
        if point.within(polygon):
            count+=1
    grid_df.at[idx, 'Traffic_Signs'] = count
    idx+=1
    
pd.set_option('display.max_rows', 200) 
grid_df

,Grid Names,West Boundary,East Boundary,North Boundary,South Boundary,geometry,Speed_Limit,Traffic_Volume,Traffic_Cameras,Traffic_Signals,Traffic_Signs
0,grid00,-114.315796,-114.270207,51.212425,51.175465,"POLYGON ((-114.31580 51.21243, -114.31580 51.1...",NaN,NaN,0.0,0.0,0.0
1,grid01,-114.315796,-114.270207,51.175465,51.138504,"POLYGON ((-114.31580 51.17546, -114.31580 51.1...",NaN,NaN,0.0,0.0,0.0
2,grid02,-114.315796,-114.270207,51.138504,51.101544,"POLYGON ((-114.31580 51.13850, -114.31580 51.1...",NaN,NaN,0.0,0.0,26.0
3,grid03,-114.315796,-114.270207,51.101544,51.064584,"POLYGON ((-114.31580 51.10154, -114.31580 51.0...",110.000000,0.0,0.0,0.0,116.0
4,grid04,-114.315796,-114.270207,51.064584,51.027624,"POLYGON ((-114.31580 51.06458, -114.31580 51.0...",NaN,NaN,0.0,0.0,0.0
5,grid05,-114.315796,-114.270207,51.027624,50.990663,"POLYGON ((-114.31580 51.02762, -114.31580 50.9...",NaN,NaN,0.0,0.0,0.0
6,grid06,-114.315796,-114.270207,50.990663,50.953703,"POLYGON ((-114.31580 50.99066, -114.31580 50.9...",NaN,NaN,0.0,0.0,0.0
7,grid07,-114.315796,-114.270207,50.953703,50.916743,"POLYGON ((-114.31580 50.95370, -114.31580 50.9...",NaN,NaN,0.0,0.0,0.0
8,grid08,-114.315796,-114.270207,50.916743,50.879782,"POLYGON ((-114.31580 50.91674, -114.31580 50.8...",NaN,NaN,0.0,0.0,0.0
9,grid09,-114.315796,-114.270207,50.879782,50.842822,"POLYGON ((-114.31580 50.87978, -114.31580 50.8...",NaN,NaN,0.0,0.0,0.0


### 2.2 Traffic Accidents Analysis
#### 2.2.1 Total (not average) number of Traffic Accidents

In [12]:
grid_df.assign(Accidents=np.nan)
idx=0
for polygon in grid_df['geometry']:
    count=0
    for long, lat in zip(incident_df['Longitude'], incident_df['Latitude']):
        point = Point(long, lat)
        if point.within(polygon):
            count+=1
    grid_df.at[idx, 'Accidents'] = count
    idx+=1

In [13]:
pd.set_option('display.max_rows', 200) 
grid_df.to_csv('Grid_DF.csv')

#### 2.2.2 Daily Weather Conditions

In [14]:
def download_weather_data(month=1, daily=True):
    """ returns a DataFrame with weather data from climate.weather.gc.ca"""
    # url string with station 51430, year of 2018 and user defined month, and daily or hourly option
    url_template = "https://climate.weather.gc.ca/climate_data/bulk_data_e.html?format=csv&stationID=50430&Year=2018&Month={month}&Day=14&timeframe={tf}&submit=Download+Data"

    if daily == False:
        tf = 1
    else: # hourly
        tf = 2
    url = url_template.format(month=month, tf = tf)

    # read data into dataframe, use headers and set Date/Time column as index
    weather_data = pd.read_csv(url, index_col='Date/Time', parse_dates=True)

    # replace the degree symbol in the column names
    weather_data.columns = [col.replace('\xb0', '') for col in weather_data.columns]
    
    return weather_data

In [15]:
daily_weather_df=pd.DataFrame()
for mo in range(1,13):
    hourly_weather_df=download_weather_data(month = mo, daily = False)
    daily_average_weather_df=hourly_weather_df.groupby(by='Day').mean()
    daily_weather_df=pd.concat([daily_weather_df, daily_average_weather_df])
pd.set_option('display.max_rows', 400) 
daily_weather_df=daily_weather_df.reset_index()
# Add Date type object to Date column
daily_weather_df['Date']=pd.to_datetime(daily_weather_df[['Day', 'Month', 'Year']]).dt.strftime('%Y-%m-%d')
daily_weather_df

,Day,Longitude (x),Latitude (y),Climate ID,Year,Month,Temp (C),Temp Flag,Dew Point Temp (C),Dew Point Temp Flag,...,Wind Spd Flag,Visibility (km),Visibility Flag,Stn Press (kPa),Stn Press Flag,Hmdx,Hmdx Flag,Wind Chill,Wind Chill Flag,Date
0,1,-114.01,51.12,3031092.0,2018.0,1.0,-16.683333,NaN,-22.166667,NaN,...,NaN,42.570833,NaN,89.715417,NaN,NaN,NaN,-21.083333,NaN,2018-01-01
1,2,-114.01,51.12,3031092.0,2018.0,1.0,-3.787500,NaN,-10.108333,NaN,...,NaN,40.891667,NaN,89.752917,NaN,NaN,NaN,-7.200000,NaN,2018-01-02
2,3,-114.01,51.12,3031092.0,2018.0,1.0,-2.391667,NaN,-7.120833,NaN,...,NaN,39.212500,NaN,89.352917,NaN,NaN,NaN,-6.647059,NaN,2018-01-03
3,4,-114.01,51.12,3031092.0,2018.0,1.0,-5.016667,NaN,-8.591667,NaN,...,NaN,40.891667,NaN,89.090000,NaN,NaN,NaN,-8.526316,NaN,2018-01-04
4,5,-114.01,51.12,3031092.0,2018.0,1.0,-0.345833,NaN,-5.733333,NaN,...,NaN,40.891667,NaN,88.583333,NaN,NaN,NaN,-10.454545,NaN,2018-01-05
5,6,-114.01,51.12,3031092.0,2018.0,1.0,3.666667,NaN,-3.354167,NaN,...,NaN,39.550000,NaN,88.210417,NaN,NaN,NaN,-4.000000,NaN,2018-01-06
6,7,-114.01,51.12,3031092.0,2018.0,1.0,-0.166667,NaN,-6.770833,NaN,...,NaN,40.020833,NaN,88.369167,NaN,NaN,NaN,-4.916667,NaN,2018-01-07
7,8,-114.01,51.12,3031092.0,2018.0,1.0,1.475000,NaN,-4.608333,NaN,...,NaN,39.212500,NaN,87.860833,NaN,NaN,NaN,-4.285714,NaN,2018-01-08
8,9,-114.01,51.12,3031092.0,2018.0,1.0,-9.854167,NaN,-12.408333,NaN,...,NaN,16.370833,NaN,87.625417,NaN,NaN,NaN,-18.416667,NaN,2018-01-09
9,10,-114.01,51.12,3031092.0,2018.0,1.0,-21.625000,NaN,-25.875000,NaN,...,NaN,10.587500,NaN,88.901667,NaN,NaN,NaN,-31.500000,NaN,2018-01-10


In [24]:
# Add weather conditions(daily average temperature and visibility) to incident_df
incident_df=incident_df.assign(Temperature=np.nan)
incident_df=incident_df.assign(Visibility=np.nan)
for date in daily_weather_df['Date']:
    index=incident_df[incident_df['Date']==date].index
    incident_df.loc[index, ['Temperature']]=daily_weather_df.iloc[row, 6]
    incident_df.loc[index, ['Visibility']]=daily_weather_df.iloc[row, -9]
    row+=1
pd.set_option('display.max_rows', 7000) 
incident_df

,START_DT,Longitude,Latitude,DateTime,Date,Time,Temperature,Visibility
0,01/01/2018 02:15:43 PM,-114.129824,51.165068,2018-01-01 14:15:43,2018-01-01,14:15,-16.683333,42.570833
1,01/01/2018 04:28:29 PM,-114.083473,51.049899,2018-01-01 16:28:29,2018-01-01,16:28,-16.683333,42.570833
2,01/01/2018 02:45:41 PM,-114.068263,51.041568,2018-01-01 14:45:41,2018-01-01,14:45,-16.683333,42.570833
3,01/01/2018 11:13:46 AM,-114.025651,50.888942,2018-01-01 11:13:46,2018-01-01,11:13,-16.683333,42.570833
4,01/01/2018 10:35:28 AM,-114.170252,51.123058,2018-01-01 10:35:28,2018-01-01,10:35,-16.683333,42.570833
5,01/01/2018 09:36:02 AM,-114.006870,51.038861,2018-01-01 09:36:02,2018-01-01,09:36,-16.683333,42.570833
6,01/01/2018 09:23:29 AM,-114.008612,51.037035,2018-01-01 09:23:29,2018-01-01,09:23,-16.683333,42.570833
7,01/01/2018 03:09:18 PM,-114.071482,51.037831,2018-01-01 15:09:18,2018-01-01,15:09,-16.683333,42.570833
8,01/01/2018 09:19:04 AM,-114.053062,51.047157,2018-01-01 09:19:04,2018-01-01,09:19,-16.683333,42.570833
9,01/01/2018 11:57:28 AM,-113.981692,51.037776,2018-01-01 11:57:28,2018-01-01,11:57,-16.683333,42.570833


#### 2.2.3 Correlation Analysis between features and Traffic Accidents

## 3. Visualization

### 3.1 Visualize the speed limit according to the roads. (5 Marks)

### 3.2 Show traffic heatmap of 2018. (5 Marks)